In [2]:

import rouge

rouge = rouge.Rouge()
# Assume that the correct modification is black -> red
hyp_list = [
    'A black fox',  # identical
    'A  black\t fox',  # identical but some format differences
    'A red fox',  # correct update
    'A cute black fox jumps over the lazy dog',  # extend only
    'A cute red fox jumps over the lazy dog',  # correct update while extend more
    'A crimson vulpine',  # Correct but close to generation instead of updating
]
ref = ['A black fox'] * len(hyp_list)

scores = rouge.get_scores(refs=ref, hyps=hyp_list)
score = scores[0]['rouge-l']['r']  # focus on the recall point

print(f"Original: {ref[0]}")
print(f"Expected: {hyp_list[2]}")
for i, score in enumerate(scores):
    hyp = hyp_list[i]
    recall = score['rouge-l']['r']
    precision = score['rouge-l']['p']
    f = score['rouge-l']['f']
    print(f'p: {precision:.2f} r: {recall:.2f} f: {f:.2f} - {hyp}')

Original: A black fox
Expected: A red fox
p: 1.00 r: 1.00 f: 1.00 - A black fox
p: 1.00 r: 1.00 f: 1.00 - A  black	 fox
p: 0.67 r: 0.67 f: 0.67 - A red fox
p: 0.33 r: 1.00 f: 0.50 - A cute black fox jumps over the lazy dog
p: 0.22 r: 0.67 f: 0.33 - A cute red fox jumps over the lazy dog
p: 0.33 r: 0.33 f: 0.33 - A crimson vulpine


In [13]:
import difflib


def calculate_diff(original: str, updated: str):
    matcher = difflib.SequenceMatcher(lambda x: x == " ", original, updated)
    print(matcher.get_opcodes())
    modifications = []
    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag != 'equal':
            # Add the modification pair (old, new) to the list
            modifications.append((original[i1:i2], updated[j1:j2]))

    return modifications


src_method = "public void process(T gray, GrayU8 binary) {\n\t\tif( verbose ) System.out.println(\"ENTER  DetectPolygonFromContour.process()\");\n\t\tInputSanityCheck.checkSameShape(binary, gray);\n\n\t\tif( imageWidth != gray.width || imageHeight != gray.height )\n\t\t\tconfigure(gray.width,gray.height);\n\n\t\t// reset storage for output. Call reset individually here to ensure that all references\n\t\t// are nulled from last time\n\t\tfor (int i = 0; i < foundInfo.size; i++) {\n\t\t\tfoundInfo.get(i).reset();\n\t\t}\n\t\tfoundInfo.reset();\n\n\t\tif( contourEdgeIntensity != null )\n\t\t\tcontourEdgeIntensity.setImage(gray);\n\n\t\tlong time0 = System.nanoTime();\n\n\t\t// find all the contours\n\t\tcontourFinder.process(binary);\n\n\t\tlong time1 = System.nanoTime();\n\n\t\t// Using the contours find the polygons\n\t\tfindCandidateShapes();\n\n\t\tlong time2 = System.nanoTime();\n\n\t\tdouble a = (time1-time0)*1e-6;\n\t\tdouble b = (time2-time1)*1e-6;\n\n\t\tmilliContour.update(a);\n\t\tmilliShapes.update(b);\n\n\t\tif( verbose ) System.out.println(\"EXIT  DetectPolygonFromContour.process()\");\n\t}"
dst_method = "public void process(T gray, GrayU8 binary) {\n\t\tif( verbose ) System.out.println(\"ENTER  DetectPolygonFromContour.process()\");\n\n\t\tif( contourPadded != null && !contourPadded.isCreatePaddedCopy() ) {\n\t\t\tint padding = 2;\n\t\t\tif( gray.width+padding != binary.width || gray.height+padding != binary.height ) {\n\t\t\t\tthrow new IllegalArgumentException(\"Including padding, expected a binary image with shape \"\n\t\t\t\t+ (gray.width+padding)+\"x\"+(gray.height+padding));\n\t\t\t}\n\t\t} else {\n\t\t\tInputSanityCheck.checkSameShape(binary, gray);\n\t\t}\n\t\tif( imageWidth != gray.width || imageHeight != gray.height )\n\t\t\tconfigure(gray.width,gray.height);\n\n\t\t// reset storage for output. Call reset individually here to ensure that all references\n\t\t// are nulled from last time\n\t\tfor (int i = 0; i < foundInfo.size; i++) {\n\t\t\tfoundInfo.get(i).reset();\n\t\t}\n\t\tfoundInfo.reset();\n\n\t\tif( contourEdgeIntensity != null )\n\t\t\tcontourEdgeIntensity.setImage(gray);\n\n\t\tlong time0 = System.nanoTime();\n\n\t\t// find all the contours\n\t\tcontourFinder.process(binary);\n\n\t\tlong time1 = System.nanoTime();\n\n\t\t// Using the contours find the polygons\n\t\tfindCandidateShapes();\n\n\t\tlong time2 = System.nanoTime();\n\n\t\tdouble a = (time1-time0)*1e-6;\n\t\tdouble b = (time2-time1)*1e-6;\n\n\t\tmilliContour.update(a);\n\t\tmilliShapes.update(b);\n\n\t\tif( verbose ) System.out.println(\"EXIT  DetectPolygonFromContour.process()\");\n\t}"

print(calculate_diff(src_method, dst_method))

candidates = [
    'Save cluster basic configuration.',
    'Save basic configuration.',
    'Save configuration.',
    'Preserve fundamental setup.',
    'Save basic configuration.',
    'Preserve and maintain the primary, fundamental setup and configuration of the computing cluster, ensuring its initial, basic settings are securely stored and kept intact.'
]

scores = rouge.get_scores(hyps=candidates,
                          refs=['@ApiOperation(value = "Save cluster basic configuration.")\n    '] * len(candidates))
for i, score in enumerate(scores):
    hyp = candidates[i]
    recall = score['rouge-l']['r']
    precision = score['rouge-l']['p']
    f = score['rouge-l']['f']
    print(f'p: {precision:.2f} r: {recall:.2f} f: {f:.2f} - {hyp}')

[('equal', 0, 126, 0, 126), ('insert', 126, 126, 126, 472), ('equal', 126, 175, 472, 521), ('replace', 175, 181, 521, 530), ('equal', 181, 1012, 530, 1361)]
[('', '\n\t\tif( contourPadded != null && !contourPadded.isCreatePaddedCopy() ) {\n\t\t\tint padding = 2;\n\t\t\tif( gray.width+padding != binary.width || gray.height+padding != binary.height ) {\n\t\t\t\tthrow new IllegalArgumentException("Including padding, expected a binary image with shape "\n\t\t\t\t+ (gray.width+padding)+"x"+(gray.height+padding));\n\t\t\t}\n\t\t} else {\n\t'), ('\n\t\tif(', '\t\t}\n\t\tif(')]
p: 0.75 r: 0.43 f: 0.55 - Save cluster basic configuration.
p: 0.67 r: 0.29 f: 0.40 - Save basic configuration.
p: 0.50 r: 0.14 f: 0.22 - Save configuration.
p: 0.00 r: 0.00 f: 0.00 - Preserve fundamental setup.
p: 0.67 r: 0.29 f: 0.40 - Save basic configuration.
p: 0.05 r: 0.14 f: 0.07 - Preserve and maintain the primary, fundamental setup and configuration of the computing cluster, ensuring its initial, basic settings